In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from keras.layers.convolutional import Conv2D
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from tensorflow.contrib.rnn import *

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
split = 7869
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [5]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [6]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [8]:
shape=(feat.shape[1], feat.shape[2])

In [10]:
MAX_SEQUENCE_LENGTH

100

In [7]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)
    # combined model
    model_combined = concatenate([model_speech, model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


model_1 = model()
model_1.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 189, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 200, 189, 1)  4           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 67, 63, 64)   640         batch_normalization_2[0][0]      
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 67, 63, 64)   0           conv2d_1[0][0]                   
___________

In [8]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit([feat[:8000],x_train_text[:8000],x_train_mocap[:8000]], vad[:8000].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate([feat[8000:],x_train_text[8000:],x_train_mocap[8000:]], vad[8000:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict([feat[6400:],x_train_text[6400:],x_train_mocap[6400:]], batch_size=32)
# np.save('speech_text_mocap-batch8000_npy', 
#          np.array(predict).reshape(3, 3639).T)


In [9]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:8000],x_train_text[:8000],x_train_mocap[:8000]],
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[8000:],x_train_text[8000:],x_train_mocap[8000:]], vad[8000:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict([feat[6400:],x_train_text[6400:],x_train_mocap[6400:]], batch_size=32)
    np.save('npy8000/stm_npy-glove-8000batch'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3639).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy8000/stm_glove_multi8000_batch-1.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 97s 15ms/step - loss: 1.5701 - v_loss: 0.6142 - a_loss: 0.4333 - d_loss: 0.5226 - v_ccc: 0.3858 - a_ccc: 0.5667 - d_ccc: 0.4774 - val_loss: 2.0912 - val_v_loss: 0.6948 - val_a_loss: 0.7423 - val_d_loss: 0.6542 - val_v_ccc: 0.3052 - val_a_ccc: 0.2577 - val_d_ccc: 0.3458
Epoch 2/50
6400/6400 [==============================] - 90s 14ms/step - loss: 1.0863 - v_loss: 0.3412 - a_loss: 0.3265 - d_loss: 0.4185 - v_ccc: 0.6588 - a_ccc: 0.6735 - d_ccc: 0.5815 - val_loss: 1.6343 - val_v_loss: 0.5996 - val_a_loss: 0.5058 - val_d_loss: 0.5289 - val_v_ccc: 0.4004 - val_a_ccc: 0.4942 - val_d_ccc: 0.4711
Epoch 3/50
6400/6400 [==============================] - 88s 14ms/step - loss: 0.9422 - v_loss: 0.2663 - a_loss: 0.3051 - d_loss: 0.3707 - v_ccc: 0.7337 - a_ccc: 0.6949 - d_ccc: 0.6293 - val_loss: 1.5368 - val_v_loss: 0

Epoch 6/50
6400/6400 [==============================] - 91s 14ms/step - loss: 0.7031 - v_loss: 0.1759 - a_loss: 0.2488 - d_loss: 0.2784 - v_ccc: 0.8241 - a_ccc: 0.7512 - d_ccc: 0.7216 - val_loss: 1.5194 - val_v_loss: 0.5510 - val_a_loss: 0.4252 - val_d_loss: 0.5433 - val_v_ccc: 0.4490 - val_a_ccc: 0.5748 - val_d_ccc: 0.4567
Epoch 7/50
6400/6400 [==============================] - 90s 14ms/step - loss: 0.6332 - v_loss: 0.1555 - a_loss: 0.2281 - d_loss: 0.2496 - v_ccc: 0.8445 - a_ccc: 0.7719 - d_ccc: 0.7504 - val_loss: 1.5596 - val_v_loss: 0.5573 - val_a_loss: 0.4351 - val_d_loss: 0.5672 - val_v_ccc: 0.4427 - val_a_ccc: 0.5649 - val_d_ccc: 0.4328
Epoch 8/50
6400/6400 [==============================] - 91s 14ms/step - loss: 0.5824 - v_loss: 0.1443 - a_loss: 0.2109 - d_loss: 0.2272 - v_ccc: 0.8557 - a_ccc: 0.7891 - d_ccc: 0.7728 - val_loss: 1.5702 - val_v_loss: 0.5610 - val_a_loss: 0.4411 - val_d_loss: 0.5681 - val_v_ccc: 0.4390 - val_a_ccc: 0.5589 - val_d_ccc: 0.4319
Epoch 9/50
6400/6400 [

Epoch 15/50
2039/2039 [==============================] - 5s 2ms/step
[0.4894315302371979, 0.5769021511077881, 0.4417027235031128]
2
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 97s 15ms/step - loss: 1.5614 - v_loss: 0.6343 - a_loss: 0.4069 - d_loss: 0.5203 - v_ccc: 0.3657 - a_ccc: 0.5931 - d_ccc: 0.4797 - val_loss: 2.2533 - val_v_loss: 0.7526 - val_a_loss: 0.7831 - val_d_loss: 0.7176 - val_v_ccc: 0.2474 - val_a_ccc: 0.2169 - val_d_ccc: 0.2824
Epoch 2/50
6400/6400 [==============================] - 91s 14ms/step - loss: 1.1042 - v_loss: 0.3764 - a_loss: 0.3158 - d_loss: 0.4121 - v_ccc: 0.6236 - a_ccc: 0.6842 - d_ccc: 0.5879 - val_loss: 1.6592 - val_v_loss: 0.6056 - val_a_loss: 0.4839 - val_d_loss: 0.5697 - val_v_ccc: 0.3944 - val_a_ccc: 0.5161 - val_d_ccc: 0.4303
Epoch 3/50
6400/6400 [==============================] - 91s 14ms/step - loss: 0.9634 - v_loss: 0.2857 - a_loss: 0.2996 - d_loss: 0.3780 - v_ccc: 0.7143 - a_ccc: 0.7004 

6400/6400 [==============================] - 91s 14ms/step - loss: 0.8609 - v_loss: 0.2420 - a_loss: 0.2818 - d_loss: 0.3372 - v_ccc: 0.7580 - a_ccc: 0.7182 - d_ccc: 0.6628 - val_loss: 1.5034 - val_v_loss: 0.5413 - val_a_loss: 0.4290 - val_d_loss: 0.5331 - val_v_ccc: 0.4587 - val_a_ccc: 0.5710 - val_d_ccc: 0.4669
Epoch 5/50
6400/6400 [==============================] - 91s 14ms/step - loss: 0.7855 - v_loss: 0.2049 - a_loss: 0.2674 - d_loss: 0.3131 - v_ccc: 0.7951 - a_ccc: 0.7326 - d_ccc: 0.6869 - val_loss: 1.5117 - val_v_loss: 0.5357 - val_a_loss: 0.4316 - val_d_loss: 0.5445 - val_v_ccc: 0.4643 - val_a_ccc: 0.5684 - val_d_ccc: 0.4555
Epoch 6/50
6400/6400 [==============================] - 92s 14ms/step - loss: 0.7060 - v_loss: 0.1822 - a_loss: 0.2465 - d_loss: 0.2773 - v_ccc: 0.8178 - a_ccc: 0.7535 - d_ccc: 0.7227 - val_loss: 1.5698 - val_v_loss: 0.5540 - val_a_loss: 0.4507 - val_d_loss: 0.5651 - val_v_ccc: 0.4460 - val_a_ccc: 0.5493 - val_d_ccc: 0.4349
Epoch 7/50
6400/6400 [===========

Epoch 12/50
6400/6400 [==============================] - 88s 14ms/step - loss: 0.4370 - v_loss: 0.1095 - a_loss: 0.1626 - d_loss: 0.1649 - v_ccc: 0.8905 - a_ccc: 0.8374 - d_ccc: 0.8351 - val_loss: 1.5794 - val_v_loss: 0.5310 - val_a_loss: 0.4648 - val_d_loss: 0.5836 - val_v_ccc: 0.4690 - val_a_ccc: 0.5352 - val_d_ccc: 0.4164
Epoch 13/50
6400/6400 [==============================] - 90s 14ms/step - loss: 0.4082 - v_loss: 0.1041 - a_loss: 0.1530 - d_loss: 0.1510 - v_ccc: 0.8959 - a_ccc: 0.8470 - d_ccc: 0.8490 - val_loss: 1.6296 - val_v_loss: 0.5614 - val_a_loss: 0.4699 - val_d_loss: 0.5982 - val_v_ccc: 0.4386 - val_a_ccc: 0.5301 - val_d_ccc: 0.4018
Epoch 14/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.3783 - v_loss: 0.0965 - a_loss: 0.1406 - d_loss: 0.1412 - v_ccc: 0.9035 - a_ccc: 0.8594 - d_ccc: 0.8588 - val_loss: 1.5835 - val_v_loss: 0.5370 - val_a_loss: 0.4632 - val_d_loss: 0.5833 - val_v_ccc: 0.4630 - val_a_ccc: 0.5368 - val_d_ccc: 0.4167
Epoch 15/50
6400/64

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 89s 14ms/step - loss: 0.4703 - v_loss: 0.1170 - a_loss: 0.1720 - d_loss: 0.1813 - v_ccc: 0.8830 - a_ccc: 0.8280 - d_ccc: 0.8187 - val_loss: 1.5466 - val_v_loss: 0.5433 - val_a_loss: 0.4360 - val_d_loss: 0.5673 - val_v_ccc: 0.4567 - val_a_ccc: 0.5640 - val_d_ccc: 0.4327
Epoch 12/50
6400/6400 [==============================] - 90s 14ms/step - loss: 0.4425 - v_loss: 0.1130 - a_loss: 0.1623 - d_loss: 0.1671 - v_ccc: 0.8870 - a_ccc: 0.8377 - d_ccc: 0.8329 - val_loss: 1.6215 - val_v_loss: 0.5361 - val_a_loss: 0.5114 - val_d_loss: 0.5740 - val_v_ccc: 0.4639 - val_a_ccc: 0.4886 - val_d_ccc: 0.4260
Epoch 13/50
6400/6400 [==============================] - 90s 14ms/step - loss: 0.4066 - v_loss: 0.1029 - a_loss: 0.1493 - d_loss: 0.1544 - v_ccc: 0.8971 - a_ccc: 0.8507 - d_ccc: 0.8456 - val_loss: 1.5600 - val_v_loss: 0.5344 - val_a_loss: 0.4532 - val_d_loss: 0.5724 - val_v_ccc: 0.4656 - val_a_ccc: 0.5468 - val_d_ccc: 0.4276
Epoch 14/50
6400/6400 [========

6400/6400 [==============================] - 86s 13ms/step - loss: 1.1154 - v_loss: 0.3763 - a_loss: 0.3205 - d_loss: 0.4185 - v_ccc: 0.6237 - a_ccc: 0.6795 - d_ccc: 0.5815 - val_loss: 1.6234 - val_v_loss: 0.5977 - val_a_loss: 0.4653 - val_d_loss: 0.5604 - val_v_ccc: 0.4023 - val_a_ccc: 0.5347 - val_d_ccc: 0.4396
Epoch 3/50
6400/6400 [==============================] - 86s 13ms/step - loss: 0.9667 - v_loss: 0.2852 - a_loss: 0.3023 - d_loss: 0.3791 - v_ccc: 0.7148 - a_ccc: 0.6977 - d_ccc: 0.6209 - val_loss: 1.5564 - val_v_loss: 0.5734 - val_a_loss: 0.4361 - val_d_loss: 0.5469 - val_v_ccc: 0.4266 - val_a_ccc: 0.5639 - val_d_ccc: 0.4531
Epoch 4/50
6400/6400 [==============================] - 87s 14ms/step - loss: 0.8726 - v_loss: 0.2394 - a_loss: 0.2842 - d_loss: 0.3490 - v_ccc: 0.7606 - a_ccc: 0.7158 - d_ccc: 0.6510 - val_loss: 1.5696 - val_v_loss: 0.5776 - val_a_loss: 0.4141 - val_d_loss: 0.5779 - val_v_ccc: 0.4224 - val_a_ccc: 0.5859 - val_d_ccc: 0.4221
Epoch 5/50
6400/6400 [===========

Epoch 9/50
6400/6400 [==============================] - 87s 14ms/step - loss: 0.5547 - v_loss: 0.1408 - a_loss: 0.1962 - d_loss: 0.2178 - v_ccc: 0.8592 - a_ccc: 0.8038 - d_ccc: 0.7822 - val_loss: 1.6093 - val_v_loss: 0.5610 - val_a_loss: 0.4551 - val_d_loss: 0.5933 - val_v_ccc: 0.4390 - val_a_ccc: 0.5449 - val_d_ccc: 0.4067
Epoch 10/50
6400/6400 [==============================] - 87s 14ms/step - loss: 0.5106 - v_loss: 0.1285 - a_loss: 0.1812 - d_loss: 0.2009 - v_ccc: 0.8715 - a_ccc: 0.8188 - d_ccc: 0.7991 - val_loss: 1.5929 - val_v_loss: 0.5323 - val_a_loss: 0.4697 - val_d_loss: 0.5909 - val_v_ccc: 0.4677 - val_a_ccc: 0.5303 - val_d_ccc: 0.4091
Epoch 11/50
6400/6400 [==============================] - 87s 14ms/step - loss: 0.4680 - v_loss: 0.1183 - a_loss: 0.1692 - d_loss: 0.1804 - v_ccc: 0.8817 - a_ccc: 0.8308 - d_ccc: 0.8196 - val_loss: 1.5588 - val_v_loss: 0.5115 - val_a_loss: 0.4620 - val_d_loss: 0.5853 - val_v_ccc: 0.4885 - val_a_ccc: 0.5380 - val_d_ccc: 0.4147
Epoch 12/50
6400/640

2039/2039 [==============================] - 5s 2ms/step
[0.5089842081069946, 0.5602083206176758, 0.4702637493610382]
18
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 102s 16ms/step - loss: 1.6106 - v_loss: 0.6212 - a_loss: 0.4468 - d_loss: 0.5426 - v_ccc: 0.3788 - a_ccc: 0.5532 - d_ccc: 0.4574 - val_loss: 2.0513 - val_v_loss: 0.7543 - val_a_loss: 0.6764 - val_d_loss: 0.6206 - val_v_ccc: 0.2457 - val_a_ccc: 0.3236 - val_d_ccc: 0.3794
Epoch 2/50
6400/6400 [==============================] - 86s 14ms/step - loss: 1.1172 - v_loss: 0.3870 - a_loss: 0.3160 - d_loss: 0.4141 - v_ccc: 0.6130 - a_ccc: 0.6840 - d_ccc: 0.5859 - val_loss: 1.7509 - val_v_loss: 0.6517 - val_a_loss: 0.4815 - val_d_loss: 0.6177 - val_v_ccc: 0.3483 - val_a_ccc: 0.5185 - val_d_ccc: 0.3823
Epoch 3/50
6400/6400 [==============================] - 87s 14ms/step - loss: 0.9631 - v_loss: 0.2879 - a_loss: 0.3007 - d_loss: 0.3746 - v_ccc: 0.7121 - a_ccc: 0.6993 - d_ccc: 0